In [2]:
!jupyter nbconvert --to script repair.ipynb
from repair import *
import pickle
f = open("test_results.pkl", "rb")
test_results = pickle.load(f)
f.close()

[NbConvertApp] Converting notebook repair.ipynb to script
[NbConvertApp] Writing 9930 bytes to repair.py


In [3]:
def side_by_side(lines1, lines2, gap=4):
    max_len = max(len(lines1), len(lines2))
    lines1 += [''] * (max_len - len(lines1))
    lines2 += [''] * (max_len - len(lines2))
    return [f"{a:<60}{' ' * gap}{b}" for a, b in zip(lines1, lines2)]
    
def side_by_side_dicts(dict1, dict2):
    for i in range(len(dict1)):
        val1 = str(dict1[i])
        val2 = str(dict2[i])
        print(f"{str(i):<6} {val1:<30}    {val2}")

def compare_structures(obj1, obj2):
    eq = True
    if isinstance(obj1, list) and isinstance(obj2, list):
        if len(obj1) != len(obj2):
            print(f"List lengths differ: {len(obj1)} vs {len(obj2)}")
            eq = False
        for i, (a, b) in enumerate(zip(obj1, obj2)):
            if a != b:
                eq = False
                print(f"List elements differ at index {i}:\n  Left:  {a}\n  Right: {b}")
        if len(obj1) != len(obj2):
            extra = obj1[len(obj2):] if len(obj1) > len(obj2) else obj2[len(obj1):]
            eq = False
            print(f"Extra elements in {'left' if len(obj1) > len(obj2) else 'right'} list: {extra}")

    elif isinstance(obj1, dict) and isinstance(obj2, dict):
        keys1 = set(obj1.keys())
        keys2 = set(obj2.keys())
        all_keys = keys1 | keys2
        for key in all_keys:
            a = obj1.get(key, None)
            b = obj2.get(key, None)
            if a != b:
                eq = False
                print(f"Dict entries differ for key {key}:\n  Left:  {a}\n  Right: {b}")
    else:
        eq = False
        print("Unsupported types for comparison. Use only lists or dicts.")
    if eq:
        print("OK")
    return eq

In [4]:
import copy

active_k_tuples_before= None 
active_k_tuples_after= None 
sequence_before= None 
sequence_after= None 
priority_queue_before= None 
priority_queue_after= None 


def debug(string, k_tuple, new_symbol):
    
    global active_k_tuples_before
    global active_k_tuples_after
    global sequence_before
    global sequence_after
    global priority_queue_before
    global priority_queue_after

    sequence, active_k_tuples = construct_active_k_tuples_and_sequence(string_to_symbol_list(string, 2)[0], 2)
    priority_queue = construct_priority_queue(active_k_tuples)

    active_k_tuples_before = copy.deepcopy(active_k_tuples)
    sequence_before = copy.deepcopy(sequence)
    priority_queue_before = copy.deepcopy(priority_queue)
    
    replace_active_k_tuple(priority_queue, active_k_tuples, sequence, k_tuple, new_symbol, 2)

    active_k_tuples_after = copy.deepcopy(active_k_tuples)
    sequence_after = copy.deepcopy(sequence)
    priority_queue_after = copy.deepcopy(priority_queue)

In [5]:
import pickle
def save_test_results(name):
    
    test_results[name] = {
        "active_k_tuples_before": active_k_tuples_before,
        "active_k_tuples_after": active_k_tuples_after,
        "sequence_before": sequence_before,
        "sequence_after": sequence_after,
        "priority_queue_before": priority_queue_before,
        "priority_queue_after": priority_queue_after,
    }
    file = open("test_results.pkl", "wb")
    pickle.dump(test_results, file)
    file.close()

In [6]:
def compare_test_results(name):

    eq = True

    print("active_k_tuples_before:")
    eq = eq and compare_structures(test_results[name]["active_k_tuples_before"],active_k_tuples_before)
    print("priority_queue_before:")
    eq = eq and compare_structures(test_results[name]["priority_queue_before"],priority_queue_before)
    print("sequence_before:")
    eq = eq and compare_structures(test_results[name]["sequence_before"],sequence_before)
    print("active_k_tuples_after:")
    eq = eq and compare_structures(test_results[name]["active_k_tuples_after"],active_k_tuples_after)
    print("sequence_after:")
    eq = eq and compare_structures(test_results[name]["sequence_after"],sequence_after)
    print("priority_queue_after:")
    eq = eq and compare_structures(test_results[name]["priority_queue_after"],priority_queue_after)

    print("-------------------------------------------------------------------------------------------")
    if eq:
        print("                                          OK                                               ")
    else:
        print("                                        ERROR                                              ")
    print("-------------------------------------------------------------------------------------------")

In [7]:
def show_test_results():
    
    print("\n" + "-" * 100)
    print("Active K-Tuples (before vs after)")
    for line in side_by_side(KTupleInfo.get_print_lines(active_k_tuples_before),KTupleInfo.get_print_lines(active_k_tuples_after)):
        print(line)

    print("\n" + "-" * 100)
    print("Sequence (before vs after)")
    for line in side_by_side(SequenceElement.get_print_lines(sequence_before), SequenceElement.get_print_lines(sequence_after)):
        print(line)
    print("-" * 100)

    print("\n" + "-" * 100)
    print("Priority queue (before vs after)")
    side_by_side_dicts(priority_queue_before, priority_queue_after)
    print("-" * 100)
	

In [8]:
debug("0101", (1,0), 3)
name = "two pairs replace middle"
show_test_results()
# save_test_results(name)
compare_test_results(name)


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (-2, 0) 1      1     
(0, 1) 2      4                                                 (1, -2) 1      5     
(1, 0) 1      3                                                 (0, 3) 1      3     
(1, -2) 1      5                                                (3, 1) 1      4     

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple                Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------              --------------------------------

In [9]:
#A and D - no old pairs elsewhere, no new pairs
debug("0123", (1,2), 4)
name = "no old pairs"
show_test_results()
# save_test_results(name)
compare_test_results(name)


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (-2, 0) 1      1     
(0, 1) 1      2                                                 (3, -2) 1      5     
(1, 2) 1      3                                                 (0, 4) 1      3     
(2, 3) 1      4                                                 (4, 3) 1      4     
(3, -2) 1      5                                                

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple                Index  Symbol     Pos    PrevKTuple   NextKTuple  
-------------------------------

In [10]:
#A and D - old pairs to left and right
debug("012323010123", (3,0), 5)
name = "old pairs left to right"
show_test_results()
# save_test_results(name)
compare_test_results(name)


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (-2, 0) 1      1     
(0, 1) 3      10                                                (0, 1) 2      10    
(1, 2) 2      11                                                (1, 2) 2      11    
(2, 3) 3      12                                                (2, 3) 2      12    
(3, 2) 1      5                                                 (3, 2) 1      5     
(3, 0) 1      7                                                 (1, 0) 1      9     
(1, 0) 1      9                                                 (3, -2) 1      13    
(3, -2) 1      13                                               (2, 5) 1      7     


In [11]:
#first old pairs replaced - updates in the .last 
debug("01232301", (3,0), 5)

name = "first old pairs replaced"
show_test_results()
# save_test_results(name)
compare_test_results(name)


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (-2, 0) 1      1     
(0, 1) 2      8                                                 (0, 1) 1      2     
(1, 2) 1      3                                                 (1, 2) 1      3     
(2, 3) 2      6                                                 (2, 3) 1      4     
(3, 2) 1      5                                                 (3, 2) 1      5     
(3, 0) 1      7                                                 (1, -2) 1      9     
(1, -2) 1      9                                                (2, 5) 1      7     
                                                                (5, 1) 1      8     


In [12]:
# ABAB - replacing BA
debug("0101", (0,1), 2)
name = "0101 replacing sides"
show_test_results()
# save_test_results(name)
compare_test_results(name)


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (2, -2) 1      5     
(0, 1) 2      4                                                 (-2, 2) 1      2     
(1, 0) 1      3                                                 (2, 2) 1      4     
(1, -2) 1      5                                                

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple                Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------              --------------------------------------------------
0

In [13]:
debug("010101", (0,1), 3)
name = "010101 replacing 01"
show_test_results()
# save_test_results(name)
compare_test_results(name)


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (3, -2) 1      7     
(0, 1) 3      6                                                 (3, 3) 2      6     
(1, 0) 2      5                                                 (-2, 3) 1      2     
(1, -2) 1      7                                                

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple                Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------              --------------------------------------------------
0

In [14]:
debug("000", (0,0), 3)

name = "interlocking 2 pairs"
show_test_results()
# save_test_results(name)
compare_test_results(name)


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (-2, 0) 1      1     
(0, 0) 2      3                                                 (0, 3) 1      3     
(0, -2) 1      4                                                (3, -2) 1      4     

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple                Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------              --------------------------------------------------
0      -2         0      -1           -1                        0 

In [15]:
debug("0000", (0,0), 3)

name = "interlocking 3 pairs"
show_test_results()
# save_test_results(name)
compare_test_results(name)
print("important! in this case we create a new active_k_tuple which is then remvoed!!!")


----------------------------------------------------------------------------------------------------
Active K-Tuples (before vs after)
Key    Count  Last                                              Key    Count  Last  
--------------------------------                                --------------------------------
(-2, 0) 1      1                                                (3, -2) 1      5     
(0, 0) 3      4                                                 (-2, 3) 1      2     
(0, -2) 1      5                                                (3, 3) 1      4     

----------------------------------------------------------------------------------------------------
Sequence (before vs after)
Index  Symbol     Pos    PrevKTuple   NextKTuple                Index  Symbol     Pos    PrevKTuple   NextKTuple  
--------------------------------------------------              --------------------------------------------------
0      -2         0      -1           -1                        0 

In [28]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("0101")[0], 2)
for x in SequenceElement.get_print_lines(s): print(x)
for x in KTupleInfo.get_print_lines(a): print(x)

TypeError: string_to_symbol_list() missing 1 required positional argument: 'k'

In [ ]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("000")[0], 2)
for x in SequenceElement.get_print_lines(s): print(x)
for x in KTupleInfo.get_print_lines(a): print(x)

In [ ]:
s, a = construct_active_k_tuples_and_sequence(string_to_symbol_list("012012")[0], 2)
SequenceElement.print_list(s)
KTupleInfo.print_dict(a)

In [69]:
sequence, active_k_tuples = construct_active_k_tuples_and_sequence(string_to_symbol_list("012012")[0], 2)
print("active_k_tuples before")
KTupleInfo.print_dict(active_k_tuples)
priority_queue = construct_priority_queue(active_k_tuples)
print("active_k_tuples after")
KTupleInfo.print_dict(active_k_tuples)
print("priority queue")
pprint(priority_queue)

active_k_tuples before
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      -1          
(1, 2) 2      5      -1          
(2, 0) 1      3      -1          
active_k_tuples after
Key    Count  Last   PosInQueue  
--------------------------------
(0, 1) 2      4      0           
(1, 2) 2      5      1           
(2, 0) 1      3      0           
priority queue
{1: [(2, 0)], 2: [(0, 1), (1, 2)]}
